In [1]:
import tensorflow as tf
from tensorflow import keras
import os, glob
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

Data sets used for this project:
- smallest dataset: 300 images (100 images/class) (/r_s_p/)
- mid-sized dataset: 7092 images (about 2500 images/class) (/hjs/test) 
- largest dataset: 300 images ( ??? images/class) (/r_c_p_3000)

#### PREPROCESSING

In [2]:
def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")  
    
    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
    target_size=(28,28)
    count=0
    for img in images:
        old_img=Image.open(img)
        
        if np.shape(old_img)[:-1] != target_size:
#             print('np.shape(old_img)[:1] ', np.shape(old_img)[:1], 'target_size ', target_size)
            new_img=old_img.resize(target_size,Image.ANTIALIAS)
            new_img.save(img, "JPEG")
            count +=1
    print(len(images), " images checked.")
    print(count, " images resized.")    

In [3]:
def load_data(img_path, number_of_data):  # 가위바위보 이미지 개수 총합에 주의하세요.  
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    return imgs, labels, idx

In [4]:
## RESIZE TRAINING DATA
classes= ['rock', 'scissor', 'paper'] ## 3 classes for this project
# 각 클래스 별로 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들이고, resize 가 필요한 파일에 대하여 resize 진행
for class_name in classes:
    image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock_scissor_paper/" + class_name
    resize_images(image_dir_path)

    print(class_name, " resizing DONE!")

3100  images checked.
0  images resized.
rock  resizing DONE!
3100  images checked.
0  images resized.
scissor  resizing DONE!
3100  images checked.
0  images resized.
paper  resizing DONE!


#### Load SEPARATE test dataset (Extra data from LMS)

In [5]:
classes= ['rock', 'scissor', 'paper']
for class_name in classes:
    image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/hjs/test/" + class_name
    resize_images(image_dir_path)

    print(class_name, " resizing DONE!")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/hjs/test"
(test_data, test_label, idx)=load_data(image_dir_path, 600)
test_data_norm = test_data/255.0   # 입력은 0~1 사이의 값으로 정규화
print("테스트 데이터(x_test)의 이미지 개수는", idx,"입니다.")

200  images checked.
0  images resized.
rock  resizing DONE!
200  images checked.
0  images resized.
scissor  resizing DONE!
200  images checked.
0  images resized.
paper  resizing DONE!
테스트 데이터(x_test)의 이미지 개수는 600 입니다.


#### BUILD MODELS

###### Set hyper-parameters

In [6]:
n_channel_1=128
n_channel_2=246
n_channel_3=512
n_dense=32
n_train_epoch=10

In [7]:
# build model with two convolution layers

model2=keras.models.Sequential()
model2.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model2.add(keras.layers.MaxPool2D(2,2))
model2.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model2.add(keras.layers.MaxPooling2D((2,2)))
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(n_dense, activation='relu'))
model2.add(keras.layers.Dense(3, activation='softmax'))

model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 246)       283638    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 246)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6150)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                196832    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9

In [8]:
# build model with three convolution layers

model3=keras.models.Sequential()
model3.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model3.add(keras.layers.MaxPool2D(2,2))
model3.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model3.add(keras.layers.MaxPooling2D((2,2)))
model3.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model3.add(keras.layers.MaxPooling2D((2,2)))
model3.add(keras.layers.Flatten())
model3.add(keras.layers.Dense(n_dense, activation='relu'))
model3.add(keras.layers.Dense(3, activation='softmax'))

model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 128)       3584      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 246)       283638    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 246)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 512)         1134080   
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 1, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)              

In [9]:
# build model with four convolution layers

model4=keras.models.Sequential()
model4.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model4.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model4.add(keras.layers.MaxPool2D(2,2))
model4.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model4.add(keras.layers.Conv2D(n_channel_3, (3,3), activation='relu'))
model4.add(keras.layers.MaxPooling2D((2,2)))
model4.add(keras.layers.Flatten())
model4.add(keras.layers.Dense(n_dense, activation='relu'))
model4.add(keras.layers.Dense(3, activation='softmax'))

model4.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 128)       3584      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 246)       283638    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 246)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 512)       1134080   
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 512)         2359808   
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 512)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)             

## using 300 imgs (my own imgs)


In [10]:
# 300 imgs Load data and generate x_train & y_train dataset
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/hjs"
(data, label, idx)=load_data(image_dir_path, 300)

# 입력은 0~1 사이의 값으로 정규화
data_norm = data/255.0

X_train, X_test, y_train, y_test = train_test_split(data_norm, label, test_size=0.2, random_state= 0)

print("총 데이터 이미지 개수는", idx,"입니다.")
print("학습 데이터(x_train)의 이미지 개수는", len(X_train),"입니다.")

총 데이터 이미지 개수는 300 입니다.
학습 데이터(x_train)의 이미지 개수는 240 입니다.


### Training

In [11]:
######### 300 training imgs & model2
model2.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model2.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
8/8 [==============================] - 6s 338ms/step - loss: 1.1437 - accuracy: 0.2957
Epoch 2/10
8/8 [==============================] - 0s 7ms/step - loss: 1.0950 - accuracy: 0.3988
Epoch 3/10
8/8 [==============================] - 0s 7ms/step - loss: 1.0871 - accuracy: 0.4227
Epoch 4/10
8/8 [==============================] - 0s 7ms/step - loss: 1.0761 - accuracy: 0.6812
Epoch 5/10
8/8 [==============================] - 0s 7ms/step - loss: 1.0537 - accuracy: 0.6172
Epoch 6/10
8/8 [==============================] - 0s 8ms/step - loss: 1.0137 - accuracy: 0.4069
Epoch 7/10
8/8 [==============================] - 0s 7ms/step - loss: 0.9441 - accuracy: 0.6948
Epoch 8/10
8/8 [==============================] - 0s 8ms/step - loss: 0.8256 - accuracy: 0.6729
Epoch 9/10
8/8 [==============================] - 0s 7ms/step - loss: 0.6848 - accuracy: 0.7612
Epoch 10/10
8/8 [==============================] - 0s 7ms/step - loss: 0.5574 - accuracy: 0.8960


In [12]:
######### 300 training imgs & model3
model3.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model3.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
8/8 [==============================] - 3s 182ms/step - loss: 1.1147 - accuracy: 0.2988
Epoch 2/10
8/8 [==============================] - 0s 11ms/step - loss: 1.0983 - accuracy: 0.3077
Epoch 3/10
8/8 [==============================] - 0s 11ms/step - loss: 1.0957 - accuracy: 0.3354
Epoch 4/10
8/8 [==============================] - 0s 11ms/step - loss: 1.0899 - accuracy: 0.3380
Epoch 5/10
8/8 [==============================] - 0s 11ms/step - loss: 1.0717 - accuracy: 0.3629
Epoch 6/10
8/8 [==============================] - 0s 11ms/step - loss: 1.0345 - accuracy: 0.5540
Epoch 7/10
8/8 [==============================] - 0s 11ms/step - loss: 0.9562 - accuracy: 0.5486
Epoch 8/10
8/8 [==============================] - 0s 11ms/step - loss: 0.8067 - accuracy: 0.7051
Epoch 9/10
8/8 [==============================] - 0s 11ms/step - loss: 0.7070 - accuracy: 0.6445
Epoch 10/10
8/8 [==============================] - 0s 11ms/step - loss: 0.5844 - accuracy: 0.7785


In [13]:
######### 300 training imgs & model4
model4.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model4.fit(data, label, epochs= n_train_epoch)

Epoch 1/10
10/10 [==============================] - 11s 609ms/step - loss: 39.7244 - accuracy: 0.3285
Epoch 2/10
10/10 [==============================] - 0s 32ms/step - loss: 1.3428 - accuracy: 0.3336
Epoch 3/10
10/10 [==============================] - 0s 32ms/step - loss: 1.1571 - accuracy: 0.3338
Epoch 4/10
10/10 [==============================] - 0s 32ms/step - loss: 1.1181 - accuracy: 0.3305
Epoch 5/10
10/10 [==============================] - 0s 32ms/step - loss: 1.1284 - accuracy: 0.4207
Epoch 6/10
10/10 [==============================] - 0s 32ms/step - loss: 1.0717 - accuracy: 0.3737
Epoch 7/10
10/10 [==============================] - 0s 31ms/step - loss: 0.9816 - accuracy: 0.5443
Epoch 8/10
10/10 [==============================] - 0s 31ms/step - loss: 0.6948 - accuracy: 0.7350
Epoch 9/10
10/10 [==============================] - 0s 31ms/step - loss: 0.5076 - accuracy: 0.8051
Epoch 10/10
10/10 [==============================] - 0s 33ms/step - loss: 0.3414 - accuracy: 0.8656


### Test

In [14]:
# split data & model2
test_loss, test_accuracy= model2.evaluate(X_test, y_test)

2/2 [==============================] - 1s 943ms/step - loss: 0.4421 - accuracy: 0.8000


In [15]:
# split data & model3
test_loss, test_accuracy= model3.evaluate(X_test, y_test)

2/2 [==============================] - 1s 565ms/step - loss: 0.6446 - accuracy: 0.6500


In [16]:
# split data & model4
test_loss, test_accuracy= model4.evaluate(X_test, y_test)

2/2 [==============================] - 2s 2s/step - loss: 1.0841 - accuracy: 0.4667


In [17]:
# Load test dataset (Extra data from LMS)
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/hjs/test"
(test_data, test_label, idx)=load_data(image_dir_path, 600)
test_data_norm = test_data/255.0   # 입력은 0~1 사이의 값으로 정규화
print("테스트 데이터(x_test)의 이미지 개수는", idx,"입니다.")

테스트 데이터(x_test)의 이미지 개수는 600 입니다.


In [18]:
# separate data & model2
test_loss, test_accuracy= model2.evaluate(test_data, test_label)

19/19 [==============================] - 1s 58ms/step - loss: 436.4802 - accuracy: 0.4150


In [19]:
# separate data & model3
test_loss, test_accuracy= model3.evaluate(test_data, test_label)

19/19 [==============================] - 1s 34ms/step - loss: 371.5153 - accuracy: 0.4117


In [20]:
# separate data & model4
test_loss, test_accuracy= model4.evaluate(test_data, test_label)

19/19 [==============================] - 2s 113ms/step - loss: 6.0279 - accuracy: 0.3917


# using 7092 imgs

In [21]:
# 7092 imgs load data and generate x_train & y_train dataset
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_tr, y_tr, idx)=load_data(image_dir_path, 7092)

x_tr_norm = x_tr/255.0   # 입력은 0~1 사이의 값으로 정규화


X_train, X_test, y_train, y_test = train_test_split(x_tr_norm, y_tr, test_size=0.2, random_state= 0)

print("총 데이터 이미지 개수는", idx,"입니다.")
print("학습 데이터(x_train)의 이미지 개수는", len(X_train),"입니다.")

총 데이터 이미지 개수는 7092 입니다.
학습 데이터(x_train)의 이미지 개수는 5673 입니다.


In [22]:
# Load test dataset
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/hjs/test"
(test_data, test_label, idx)=load_data(image_dir_path, 600)
test_data_norm = test_data/255.0   # 입력은 0~1 사이의 값으로 정규화
print("테스트 데이터(x_trest)의 이미지 개수는", idx,"입니다.")

테스트 데이터(x_trest)의 이미지 개수는 600 입니다.


### Training

In [23]:
######### 7092 training imgs & model2
model2.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model2.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
178/178 [==============================] - 4s 22ms/step - loss: 1.1677 - accuracy: 0.4134
Epoch 2/10
178/178 [==============================] - 1s 7ms/step - loss: 0.7218 - accuracy: 0.7034
Epoch 3/10
178/178 [==============================] - 1s 7ms/step - loss: 0.3896 - accuracy: 0.8587
Epoch 4/10
178/178 [==============================] - 1s 7ms/step - loss: 0.2032 - accuracy: 0.9348
Epoch 5/10
178/178 [==============================] - 1s 7ms/step - loss: 0.1221 - accuracy: 0.9664
Epoch 6/10
178/178 [==============================] - 1s 7ms/step - loss: 0.0932 - accuracy: 0.9753
Epoch 7/10
178/178 [==============================] - 1s 7ms/step - loss: 0.1743 - accuracy: 0.9460
Epoch 8/10
178/178 [==============================] - 1s 7ms/step - loss: 0.0376 - accuracy: 0.9921
Epoch 9/10
178/178 [==============================] - 1s 7ms/step - loss: 0.0330 - accuracy: 0.9905
Epoch 10/10
178/178 [==============================] - 1s 7ms/step - loss: 0.0178 - accuracy: 0.997

In [24]:
######### 7092 training imgs & model3
model3.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model3.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
178/178 [==============================] - 4s 21ms/step - loss: 1.1770 - accuracy: 0.4252
Epoch 2/10
178/178 [==============================] - 2s 11ms/step - loss: 0.6919 - accuracy: 0.6956
Epoch 3/10
178/178 [==============================] - 2s 11ms/step - loss: 0.4250 - accuracy: 0.8262
Epoch 4/10
178/178 [==============================] - 2s 11ms/step - loss: 0.3019 - accuracy: 0.8883
Epoch 5/10
178/178 [==============================] - 2s 11ms/step - loss: 0.1680 - accuracy: 0.9457
Epoch 6/10
178/178 [==============================] - 2s 11ms/step - loss: 0.1157 - accuracy: 0.9596
Epoch 7/10
178/178 [==============================] - 2s 12ms/step - loss: 0.1014 - accuracy: 0.9677
Epoch 8/10
178/178 [==============================] - 2s 11ms/step - loss: 0.0615 - accuracy: 0.9824
Epoch 9/10
178/178 [==============================] - 2s 11ms/step - loss: 0.0586 - accuracy: 0.9813
Epoch 10/10
178/178 [==============================] - 2s 11ms/step - loss: 0.0457 - accura

In [25]:
######### 7092 training imgs & model4
model4.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model4.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
178/178 [==============================] - 12s 67ms/step - loss: 1.0718 - accuracy: 0.4308
Epoch 2/10
178/178 [==============================] - 6s 33ms/step - loss: 0.7955 - accuracy: 0.6553
Epoch 3/10
178/178 [==============================] - 6s 33ms/step - loss: 0.3776 - accuracy: 0.8527
Epoch 4/10
178/178 [==============================] - 6s 33ms/step - loss: 0.2061 - accuracy: 0.9306
Epoch 5/10
178/178 [==============================] - 6s 33ms/step - loss: 0.1127 - accuracy: 0.9645
Epoch 6/10
178/178 [==============================] - 6s 33ms/step - loss: 0.0593 - accuracy: 0.9816
Epoch 7/10
178/178 [==============================] - 6s 33ms/step - loss: 0.0238 - accuracy: 0.9926
Epoch 8/10
178/178 [==============================] - 6s 33ms/step - loss: 0.0158 - accuracy: 0.9960
Epoch 9/10
178/178 [==============================] - 6s 33ms/step - loss: 0.1109 - accuracy: 0.9687
Epoch 10/10
178/178 [==============================] - 6s 33ms/step - loss: 0.0206 - accur

## Test

In [26]:
# split data & model2
test_loss, test_accuracy= model2.evaluate(X_test, y_test)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

45/45 [==============================] - 2s 36ms/step - loss: 0.1206 - accuracy: 0.9591
test_loss: 0.12056318670511246 
test_accuracy: 0.9591261744499207


In [27]:
# split data & model3
test_loss, test_accuracy= model3.evaluate(X_test, y_test)

45/45 [==============================] - 1s 22ms/step - loss: 0.0677 - accuracy: 0.9789


In [28]:
# split data & model4
test_loss, test_accuracy= model4.evaluate(X_test, y_test)

45/45 [==============================] - 3s 64ms/step - loss: 0.0309 - accuracy: 0.9887


In [29]:
# separate data & model2
test_loss, test_accuracy= model2.evaluate(test_data, test_label)

19/19 [==============================] - 0s 4ms/step - loss: 420.6739 - accuracy: 0.4883


In [30]:
# separate data & model3
test_loss, test_accuracy= model3.evaluate(test_data, test_label)

19/19 [==============================] - 0s 5ms/step - loss: 503.0591 - accuracy: 0.4933


In [31]:
# separate data & model4
test_loss, test_accuracy= model4.evaluate(test_data, test_label)

19/19 [==============================] - 0s 11ms/step - loss: 364.4283 - accuracy: 0.5583


### 16392   DATA

In [32]:
## 16392  imgs load data and generate x_train & y_train dataset
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_tr, y_tr, idx)=load_data(image_dir_path, 7092)

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock_scissor_paper/" + class_name
(x_tr_1, y_tr_1, idx)=load_data(image_dir_path, 9300)

x_tr_data= np.vstack((x_tr, x_tr_1)) ## total 13113 images
y_tr_data= np.append(y_tr, y_tr_1)

x_tr_data_norm = x_tr_data/255.0   # 입력은 0~1 사이의 값으로 정규화

X_train, X_test, y_train, y_test = train_test_split(x_tr_data_norm, y_tr_data, test_size=0.2, random_state= 0)

print("총 데이터 이미지 개수는", len(y_tr_data) ,"입니다.")
print("학습 데이터(x_train)의 이미지 개수는", len(X_train),"입니다.")
print("test 데이터(x_test)의 이미지 개수는", len(X_test),"입니다.")

총 데이터 이미지 개수는 16392 입니다.
학습 데이터(x_train)의 이미지 개수는 13113 입니다.
test 데이터(x_test)의 이미지 개수는 3279 입니다.


In [33]:
# Load test dataset
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/hjs/test"
(test_data, test_label, idx)=load_data(image_dir_path, 600)
test_data_norm = test_data/255.0   # 입력은 0~1 사이의 값으로 정규화
print("테스트 데이터(x_trest)의 이미지 개수는", idx,"입니다.")

테스트 데이터(x_trest)의 이미지 개수는 600 입니다.


### Training

In [34]:
######### 16392 training imgs & model2
model2.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model2.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
410/410 [==============================] - 7s 16ms/step - loss: 0.1118 - accuracy: 0.9702
Epoch 2/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0164 - accuracy: 0.9960
Epoch 3/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0208 - accuracy: 0.9941
Epoch 4/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0458 - accuracy: 0.9857
Epoch 5/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0166 - accuracy: 0.9951
Epoch 6/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0051 - accuracy: 0.9988
Epoch 7/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0223 - accuracy: 0.9927
Epoch 8/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0134 - accuracy: 0.9964
Epoch 9/10
410/410 [==============================] - 3s 7ms/step - loss: 0.0037 - accuracy: 0.9992
Epoch 10/10
410/410 [==============================] - 3s 7ms/step - loss: 8.3494e-04 - accuracy: 0

In [35]:
######### 16392 training imgs & model3
model3.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model3.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
410/410 [==============================] - 7s 17ms/step - loss: 0.0443 - accuracy: 0.9848
Epoch 2/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0370 - accuracy: 0.9862
Epoch 3/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0407 - accuracy: 0.9880
Epoch 4/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0216 - accuracy: 0.9932
Epoch 5/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0361 - accuracy: 0.9880
Epoch 6/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0129 - accuracy: 0.9960
Epoch 7/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0376 - accuracy: 0.9881
Epoch 8/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0038 - accuracy: 0.9994
Epoch 9/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0101 - accuracy: 0.9973
Epoch 10/10
410/410 [==============================] - 5s 11ms/step - loss: 0.0228 - accura

In [36]:
######### 16392 training imgs & model4
model4.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model4.fit(X_train, y_train, epochs= n_train_epoch)

Epoch 1/10
410/410 [==============================] - 22s 53ms/step - loss: 0.1307 - accuracy: 0.9803
Epoch 2/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0080 - accuracy: 0.9970
Epoch 3/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0218 - accuracy: 0.9939
Epoch 4/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0217 - accuracy: 0.9942
Epoch 5/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0099 - accuracy: 0.9968
Epoch 6/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0144 - accuracy: 0.9976
Epoch 7/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0088 - accuracy: 0.9979
Epoch 8/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0134 - accuracy: 0.9965
Epoch 9/10
410/410 [==============================] - 14s 33ms/step - loss: 0.0064 - accuracy: 0.9984
Epoch 10/10
410/410 [==============================] - 14s 33ms/step - loss: 0.001

## Test

In [37]:
# split data & model2
test_loss, test_accuracy= model2.evaluate(X_test, y_test)

103/103 [==============================] - 3s 24ms/step - loss: 0.0124 - accuracy: 0.9970


In [38]:
# split data & model3
test_loss, test_accuracy= model3.evaluate(X_test, y_test)

103/103 [==============================] - 2s 14ms/step - loss: 0.0269 - accuracy: 0.9939


In [39]:
# split data & model4
test_loss, test_accuracy= model4.evaluate(X_test, y_test)

103/103 [==============================] - 4s 41ms/step - loss: 0.0113 - accuracy: 0.9970


In [40]:
# separate data & model2
test_loss, test_accuracy= model2.evaluate(test_data, test_label)

19/19 [==============================] - 0s 4ms/step - loss: 673.7531 - accuracy: 0.4700


In [41]:
# separate data & model3
test_loss, test_accuracy= model3.evaluate(test_data, test_label)

19/19 [==============================] - 0s 5ms/step - loss: 214.3754 - accuracy: 0.6583


In [42]:
# separate data & model4
test_loss, test_accuracy= model4.evaluate(test_data, test_label)

19/19 [==============================] - 0s 11ms/step - loss: 295.4477 - accuracy: 0.5717


## 7092 raw data (not normalized)

In [43]:
### 정규화 하지 않은 데이터 생성
X_train, X_test, y_train, y_test = train_test_split(x_tr, y_tr, test_size=0.2, random_state= 0)

print("학습 데이터(x_train)의 이미지 개수는", len(X_train),"입니다.")

# SEPARATE test dataset 도 정규화 하지 않은 데이터 생성
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/hjs/test"
(test_data, test_label, idx)=load_data(image_dir_path, 600)
print("테스트 데이터(x_trest)의 이미지 개수는", idx,"입니다.")

학습 데이터(x_train)의 이미지 개수는 5673 입니다.
테스트 데이터(x_trest)의 이미지 개수는 600 입니다.


In [44]:
model2.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model2.fit(x_tr, y_tr, epochs= 10)

Epoch 1/10
222/222 [==============================] - 6s 27ms/step - loss: 10.7833 - accuracy: 0.9822
Epoch 2/10
222/222 [==============================] - 2s 7ms/step - loss: 5.6605 - accuracy: 0.9855
Epoch 3/10
222/222 [==============================] - 2s 7ms/step - loss: 2.8481 - accuracy: 0.9877
Epoch 4/10
222/222 [==============================] - 2s 7ms/step - loss: 0.8657 - accuracy: 0.9933
Epoch 5/10
222/222 [==============================] - 2s 7ms/step - loss: 0.9001 - accuracy: 0.9927
Epoch 6/10
222/222 [==============================] - 2s 7ms/step - loss: 0.6314 - accuracy: 0.9959
Epoch 7/10
222/222 [==============================] - 2s 7ms/step - loss: 0.1932 - accuracy: 0.9978
Epoch 8/10
222/222 [==============================] - 2s 7ms/step - loss: 0.6663 - accuracy: 0.9929
Epoch 9/10
222/222 [==============================] - 2s 7ms/step - loss: 3.1187 - accuracy: 0.9804
Epoch 10/10
222/222 [==============================] - 2s 7ms/step - loss: 2.8275 - accuracy: 0.98

In [45]:
######### 7092 training imgs - not normalized
model3.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model3.fit(x_tr, y_tr, epochs= 10)

Epoch 1/10
222/222 [==============================] - 6s 24ms/step - loss: 14.6525 - accuracy: 0.9661
Epoch 2/10
222/222 [==============================] - 3s 11ms/step - loss: 2.6326 - accuracy: 0.9887
Epoch 3/10
222/222 [==============================] - 3s 11ms/step - loss: 1.1374 - accuracy: 0.9902
Epoch 4/10
222/222 [==============================] - 3s 11ms/step - loss: 0.7465 - accuracy: 0.9927
Epoch 5/10
222/222 [==============================] - 3s 11ms/step - loss: 0.2100 - accuracy: 0.9942
Epoch 6/10
222/222 [==============================] - 3s 11ms/step - loss: 0.9872 - accuracy: 0.9892
Epoch 7/10
222/222 [==============================] - 3s 11ms/step - loss: 0.7818 - accuracy: 0.9919
Epoch 8/10
222/222 [==============================] - 3s 11ms/step - loss: 0.0316 - accuracy: 0.9968
Epoch 9/10
222/222 [==============================] - 3s 12ms/step - loss: 0.0198 - accuracy: 0.9985
Epoch 10/10
222/222 [==============================] - 3s 11ms/step - loss: 0.0044 - accur

In [46]:
######### 7092 training imgs - not normalized
model4.compile(optimizer='adam',
                 loss= 'sparse_categorical_crossentropy',
                 metrics=['accuracy'])
model4.fit(x_tr, y_tr, epochs= 10)

Epoch 1/10
222/222 [==============================] - 17s 74ms/step - loss: 7.0935 - accuracy: 0.9839
Epoch 2/10
222/222 [==============================] - 7s 33ms/step - loss: 1.5959 - accuracy: 0.9922
Epoch 3/10
222/222 [==============================] - 7s 33ms/step - loss: 0.7760 - accuracy: 0.9957
Epoch 4/10
222/222 [==============================] - 7s 33ms/step - loss: 0.2619 - accuracy: 0.9977
Epoch 5/10
222/222 [==============================] - 7s 33ms/step - loss: 1.0789 - accuracy: 0.9938
Epoch 6/10
222/222 [==============================] - 7s 33ms/step - loss: 1.0663 - accuracy: 0.9901
Epoch 7/10
222/222 [==============================] - 7s 33ms/step - loss: 0.8592 - accuracy: 0.9873
Epoch 8/10
222/222 [==============================] - 7s 33ms/step - loss: 0.1996 - accuracy: 0.9976
Epoch 9/10
222/222 [==============================] - 7s 33ms/step - loss: 0.1852 - accuracy: 0.9954
Epoch 10/10
222/222 [==============================] - 7s 33ms/step - loss: 0.0024 - accur

In [47]:
# split data & model2
test_loss, test_accuracy= model2.evaluate(X_test, y_test)

45/45 [==============================] - 0s 4ms/step - loss: 0.5683 - accuracy: 0.9930


In [48]:
# split data & model3
test_loss, test_accuracy= model3.evaluate(X_test, y_test)

45/45 [==============================] - 0s 5ms/step - loss: 1.8903e-05 - accuracy: 1.0000


In [49]:
# split data & model4
test_loss, test_accuracy= model4.evaluate(X_test, y_test)

45/45 [==============================] - 1s 11ms/step - loss: 0.5215 - accuracy: 0.9887


In [50]:
# separate data & model2
test_loss, test_accuracy= model2.evaluate(test_data, test_label)

19/19 [==============================] - 0s 4ms/step - loss: 190.0203 - accuracy: 0.5383


In [51]:
# separate data & model3
test_loss, test_accuracy= model3.evaluate(test_data, test_label)

19/19 [==============================] - 0s 5ms/step - loss: 66.8886 - accuracy: 0.6583


In [52]:
# separate data & model4
test_loss, test_accuracy= model4.evaluate(test_data, test_label)

19/19 [==============================] - 0s 11ms/step - loss: 75.7625 - accuracy: 0.5550


### 회고

#### Dataset
- 가위, 바위, 보 3가지의 클래스를 분류하는 모델에 대하여, 3가지 종류의 데이터를 사용하여 각각 학습하였다. 
- Training Dataset: 
    각 데이터셋은 이미지 갯수에 차이가 있으며, 중복된 데이터가 있을 가능성이 있다.
    - smallest dataset: 300 images: 내가 찍은 각 100장, 총 300장의 이미지를 사용하였다.
    - mid-sized dataset: 7092 images: 각 학생들이 올려 공유한 데이터 중 일부를 통합하여 사용하였다.
    - largest dataset: 16392 images: 모든 사람들이 올리고 공유한 모든 데이터를 통합하여 사용하였다.
- Test Dataset:
    전체 데이터를 training & test dataset 으로 나누어 test data을 확보하였으나, 여기서 확보한 test dataset 은 사실 이미지를 얻는 과정에서 웹캠이 빠른 속도로 찍으면서 중복되거나 아주 비슷한 이미지가 여러개 있을 가능성이 있으므로 training data 와 중복된 데이터가 포함될 가능성이 높다.
    학습 된 모델의 실제 성능을 검증하기 위해서는 모델이 보지 못한 데이터로 테스트 하는것이 원칙이므로, 이 데이터셋 으로는 제대로 된 성능을 검증하기 어려울 수 있다고 판단하여 training data 에 사용하지 않은 새로운 데이터셋을 테스트 데이터셋으로 사용하였다.(이 때에는 LMS sample 에서 얻은 각 클래스별로 200장, 총 600장의 데이터셋을 사용하였다.)
    - 아래 결과 테이블 에서는 split 하여 얻은 test data와 LMS sample 에서 얻은 test data 두가지로 나누어 시험한 것을 볼 수 있다.
    
    
#### Preprocessing
- 모든 이미지를 28x28x3 으로 resize 하여 사용하였다.
- 모든 이미지를 255 로 나누어 normalize 하여 사용하였다.


#### Hyperparameter
- 컨볼루션 레이어의 갯수를 2, 3, 4 로 키워 학습시켜보았으며, 각 컨볼루션 레이어의 hyperparameter 를 128, 64, 32, 16 으로 세팅, dense layer 는 32, 마지막 dense layer 는 클래스의 갯수에 맞추어 3으로 맞춰주었다.
- 각 모델별 epoch 수는 3, 5, 7, 10 으로 학습시켜 비교하였다.

(validation - accuracy 가 안나와 cross-validation 을 해보고자 StratifiedKFold 를 사용하였으며, split 갯수는 3, 4, 5 로 테스트 하였다.


#### Evaluation
- 300개의 이미지만 사용하였을 때에는 split 데이터 사용시 최고성능 88.33%, 새로운 데이터 사용시 최고성능 41.5%로 성능이 매우 낮음을 볼 수 있다. 
- 7092개의 이미지를 사용하여 학습시켰을 때에는 split 데이터로 test 시 최고성능 98%, 새로운 데이터로 test 시 최고성능 55.8% 로, 300개로 학습시켰을 때 보다 각 모델 별 혹은 test data별로 10~20% 정도의 성능 향상이 있었다.
- 16392개의 이미지를 사용하여 학습 시켰을 때에는 7092 개의 데이터를 사용하였을 보다 조금 높아지며, 3 레이어를 사용하였 을 때 65.83% 로 성능 향상이 있었다. 
- 큰 데이터셋으로 학습한 모델들이 새로운 데이터셋으로 테스트 했을 때, accuracy가 높아졌지만, 이와 동시에 loss 값이 함께 올라갔다. 

##### Result
실험 결과를 통해
1. split data 로 test 했을 때에는 90% 이상이 나오지만 separate 데이터를 사용하였을 때에는 40~50% 인 것으로 보아 우리가 사용한 데이터에는 비슷한 데이터가 많아 train_test_split 을 사용하여 나눈 test data 는 train data 와 중복성이 있을 것으로 예상할 수 있다.
2. 데이터를 많이 사용하였을 때 데이터가 적을 때 보다 모델 성능이 향상됨을 볼 수 있다.
3. 모델의 성능을 안정적으로 높이기 위해서는 loss 값을 줄이기 위한 다른 방향을 찾아봐야 하겠다.


|     No. of Data   |  No. of Conv Layers|  split | separate |                                  Remarks                                  |
|:-----------------:|:------------------:|:------:|:--------:|:--------------------------------------------------------------------------|
|        300        |          2d        | 0.8    | 0.415    | > 새로운 데이터에 대해서 성능이 매우 낮음                 |
|                   |          3d        | 0.65   | 0.4117   |                                                                           |
|                   |          4d        | 0.4667 | 0.3917   |                                                                           |
|        7092       |          2d        | 0.9591 | 0.4883   | > 300개 이미지로만 했을때와 비교하면  기존 데이터를 나눈 경우에도 95% 이상으로 성능이|
|                   |          3d        | 0.9789 | 0.4933   |   매우 높고 새로운 데이터로 했을 때에도 10% 정도의 성능 향상이 있음.         |
|                   |          4d        | 0.9887 | 0.5583   |                   |
|       16392       |          2d        | 0.997  | 0.47     | > 7092 데이터로 했을때와 마찬가지로 split 한 데이터로 테스트 했을 때  99% 이상의 성능이 |
|                   |          3d        | 0.9939 | 0.6583   |           나왔고 새로운 데이터로 테스트 했을 때에는 7092개로 했을 때 보다 조금씩 더 올랐다.|
|                   |          4d        | 0.997  | 0.5717   | |

|   Hyper Parameters Used  |
|:------------------------:|
|     n_channel_1 = 128    |
|     n_channel_2 = 246    |
|     n_channel_3 = 512    |
|       n_dense = 32       |
|    n_train_epoch = 10    |
| train & test split : 20% |
|     Random state = 0     |